Sonnet Generator
- Rudransh Shinghal
- 102065029

In [1]:
import random
from bs4 import BeautifulSoup
import requests
from Phyme import Phyme
import re
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pickle

In [2]:
def gen_rhyme_scheme():
    types = ['ABAB CDCD EFEF GG','ABBA ABBA CDCD CD','ABBA ABBA CDEC DE','ABAB BCBC CDCD EE',
            'ABA BCB CDC DAD AA']
    return random.choice(types)

In [3]:
def find_syn(word,time = 8):
    if time <= 0:
        return set()
    print(time)
    url = f"https://www.thesaurus.com/browse/{word}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    ul = soup.find('div', attrs={'data-testid': 'word-grid-container'}).find('ul').find_all('a')
    syns = set()
    for u in ul[:7]:
        syns.add(u.text.strip())
    new_set = find_syn(random.choice(list(syns)),time-1)
    syns = syns.union(new_set)
    syns = list(syns)
    syns.insert(0,word)
    return syns

In [22]:
def get_rhyme(word,number = 2):
    ph = Phyme()
#     ry = ph.get_perfect_rhymes(word)
    try:
        ry = ph.get_perfect_rhymes(word)
    except:
        return []
    words = [word]
#     words.append(word)
    for key in ry:
        for k in ry[key]:
            w = res = re.sub(r'[^a-zA-Z]', '', k)
            if w not in words:
                words.append(w)
    return words[:number]

In [5]:
get_rhyme('crime',5)

['crime', 'im', 'time', 'climb', 'grime']

In [6]:
meta_data = [('models/simile_generator.h5','tokenizers/simle_tokenizer.pickle',22),
       ('models/metaphor_generator.h5','tokenizers/meta_tokenizer.pickle',57),
       ('models/personification_generator.h5','tokenizers/per_tokenizer.pickle',50),
       ('models/allitration_generator.h5','tokenizers/al_tokenizer.pickle',26),
        ('models/norm_generator.h5','tokenizers/norm_tokenizer.pickle',27)
       ] 

In [7]:
def best_words():
    s = set()
    i = 0
    for data in meta_data:
        with open(data[1], 'rb') as handle:
            token = pickle.load(handle)
        if i == 0:
            s = s.union(set(list(token.word_index.keys())))
        else:
            s = s.intersection(set(list(token.word_index.keys())))
        i+=1
    return list(s)

In [8]:
def get_model_data(data):
    model = keras.models.load_model(data[0])
    with open(data[1], 'rb') as handle:
        token = pickle.load(handle)
    return model,token,data[2]

In [9]:
def predict_words(seed, no_words,n):
    model,tokenizer,max_seq_length = get_model_data(meta_data[n])
    for i in range(no_words):
        token_list = tokenizer.texts_to_sequences([seed])[0]
        token_list = pad_sequences([token_list], maxlen=max_seq_length-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=1)
        for word, index in tokenizer.word_index.items():
            if predicted == index:
                new_word = word
                break
        seed += " " + new_word

        if(i==no_words-1):
            return seed

In [10]:
def make_new_sentence(model_num,n,word):
    seed_text = word
    next_words = n
    sentence = predict_words(seed_text, next_words,model_num)
    words = sentence.split()
    reversed_sentence = " ".join(words[::-1])
    return reversed_sentence

In [28]:
def make_poem_data(topic):
    rhy = list(gen_rhyme_scheme())
    syns = find_syn(topic,5)
    best = random.sample(best_words(),10)
    final_words = {}
    i,j = 0,0
    for r in set(rhy):
        if r!= ' ':
            word_list,z = (syns,i) if random.randint(0,1) == 0 else (best,j)
            count = len([c for c in rhy if c==r])
            word = word_list[z]
            rhyms = get_rhyme(word,count)
            while(len(rhyms) < count):
                z+=1
                word = word_list[z]
                rhyms = get_rhyme(word,count)
            z+=1
            i = z if word_list == syns else i
            j = z if word_list == best else j
            final_words[r] = rhyms
    return rhy,final_words


In [58]:
def most_common(lst):
    return max(set(lst), key=lst.count)
def make_poem(topic):
    rhy,final_words = make_poem_data(topic)
    poem = []
    prev_model = 0
    used = [0]
    for r in rhy:
        if(r != ' '):
            sen_len = random.randint(4,10)
            model_num = random.randint(0,4)
            while model_num==prev_model or model_num== most_common(used):
                model_num = random.randint(0,4)
#             print(model_num,prev_model,used)
            word = random.choice(final_words[r])
            final_words[r].remove(word)
            sentence = make_new_sentence(model_num,sen_len,word) 
            poem.append(sentence)
            prev_model = model_num
            used.append(model_num)
        else:
            poem.append('\n')
    return poem

In [59]:
poem = make_poem('love')

5
4
3
2
1
1/1 [==============================] - 0s 25ms/step


In [60]:
for p in poem:
    print(p)

of the happy opn parkin
the elephant voice rocks but a beautiful juice section
his legions ithuriel with unvisited of the happy opn givin


fanatic followed the earths exquisite affection
young encouraging them to walk and a necessary dove
legions ithuriel with unvisited of the happy opn flexion


is drowning in the sea of love
thats beautiful drifts away like the shadows
something to be allergic to shove


when crawled mingling are like building castles in the kilos
found fertile soil to flourish in
that gathered wild rice in the meadows


a necessary evil pushing me to be been
waddled we fiona the football fanatic followed the earths exquisite tellin


In [61]:
poem = make_poem('hate')

5
4
3
2
1
1/1 [==============================] - 0s 29ms/step


In [62]:
for p in poem:
    print(p)

and slunk dipt bending each the birds of third and enmity
analyzed and interpreted the soul
encouraging them to walk and a necessary purity
to be allergic to roll


the rabbit oer the bitterness
building castles in the bohrer
her young encouraging them to walk and a necessary tenderness
with feathers that perches in the keys to so horror


puzzle each piece fitting together in its own unique way
the drums of love is something to be allergic to strait
down and the moon turned over to face the day
so heavnly love shal outdoo hellish hate


crawled mingling are like building castles in the racism
ithuriel with unvisited of the happy opn antagonism
